# Sea-level response from water mass inputs

This interactive notebook enables you to explore how changes in terrestrial water mass, such as from ice-sheet melt, affect global and regional sea level.

**Key features:**
- **Dataset selection:** Choose from provided datasets for ice-mass input from Greenland and/or Antarctica, or upload your own mass balance data.
- **Flexible time period:** Specify the start and end years for your analysis.
- **Scenario exploration:** Apply custom scaling factors to simulate different "what-if" scenarios of ice melt or water mass change.
- **Visualization:** Instantly view the global sea-level response from the input water masses.
- **Location-specific analysis:** Enter coordinates to investigate sea-level change at any point on the globe.

**How to use this notebook:**
1. Select or upload a mass balance dataset.
2. Choose your analysis period.
3. Adjust the scaling factor to explore different scenarios.
4. Visualize the resulting sea-level response globally or at specific locations.

**Requirements:**  
- This notebook is designed for use in Jupyter (or VS Code with the Jupyter extension).
- No prior coding experience is required; all interactions are via user-friendly widgets.

---

*For more details on the datasets and methods, see the documentation or code comments below.*

In [ ]:
import sys

print("⏳ Installing dependencies...")
if "google.colab" in sys.modules:
    %pip install -q xarray zarr cartopy s3fs==2025.3.0
    %pip install --no-deps -q git+https://github.com/DTC-Ice-Sheets/dtc_is_notebooks.git


## Notebook preparations

In [ ]:
import os
from datetime import datetime

import ipywidgets as widgets
import xarray as xr
from IPython.display import clear_output
from matplotlib.widgets import Button

from dtc_is_notebook_helpers.api_helpers import (
    get_precomputed_mass_balance_dataset_url,
    run_selrem_module,
    upload_mass_balance_csv,
)
from dtc_is_notebook_helpers.uc2_plotting_helpers import (
    MASS_BALANCE_COL_NAME,
    MASS_BALANCE_ERROR_COL_NAME,
    compute_mean_mass_balance_over_time_window,
    plot_global_slr_three_panels,
    plot_scaled_mean_mass_balance,
    plot_slr_location_four_panels,
)

os.environ["OMP_DISPLAY_ENV"] = "FALSE"
os.environ["KMP_WARNINGS"] = "FALSE"
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

api_password = input("Please enter DTC-IS API password: ")
os.environ["DTC_API_PASSWORD"] = api_password

## Select mass input dataset

You can select which mass input dataset to use for your analysis:

- **Provided datasets:** Choose from pre-loaded datasets representing mass input from 
  - Greenland Ice Sheet (GrIS), available between 1992 and 2018. Derived from radar altimetry. [Link to article.](https://doi.org/10.1029/2020GL091216)
  - Antarctic Ice Sheet (AIS), available between 2003 and 2024. Derived from GRACE/GRACE-FO measurements. [Link to data portal.](https://doi.org/10.5880/COST-G.GRAVIS_01_L3_ICE)
  - or a combination of both. available between 2003 and 2018.
- **Upload your own:** Alternatively, you can upload your own gridded mass balance dataset. Your file should be in CSV format, with columns for latitude, longitude, annual mass balance (in kg/yr), and associated uncertainty for each year.

**CSV file format example:**

| lat        | lon        | 1993_mb   | 1993_mb_error | 1994_mb   | 1994_mb_error | 1995_mb   | 1995_mb_error |
|------------|------------|-----------|---------------|-----------|---------------|-----------|---------------|
| 60.526234  | -44.33303  | -1.16E+10 | 8.92E+08      | -1.16E+10 |               | -9.27E+09 | 8.95E+08      |
| 60.569984  | -44.419952 | -1.06E+10 | 8.90E+08      | -1.05E+10 |               | -6.94E+09 | 8.94E+08      |
| 60.569515  | -44.33201  | -7.91E+09 | 8.91E+08      | -7.59E+09 |               | -5.89E+09 | 8.94E+08      |

- Each row represents a grid cell with its latitude and longitude.
- For each year, provide both the mass balance (`*_mb`) and its uncertainty (`*_mb_error`).
- It is possible to leave the error columns empty, as long as the header still matches (e.g., 1994).
- Units for mass balance are **kg/yr**.

---

In [ ]:

dataset_options = [
    ("Mass input from GrIS", "GrIS"),
    ("Mass input from AIS", "AIS"),
    ("Mass input from both AIS and GrIS", "AIS and GrIS"),
    ("Upload own mass input", "custom")
]
dataset_dropdown = widgets.Dropdown(
    options=dataset_options,
    value="GrIS",
    description="Dataset:",
    style={"description_width": "initial"}
)
start_year_widget = widgets.IntText(value=2010, description="Start year:", style={"description_width": "initial"})
end_year_widget = widgets.IntText(value=2018, description="End year:", style={"description_width": "initial"})
file_upload = widgets.FileUpload(accept=".csv", multiple=False, description="Upload file")
submit_button = widgets.Button(description="Submit", button_style="success")
selection_output = widgets.Output()
status_label = widgets.Label(value="")
submit_button.disabled = False

def on_dataset_change(change: dict) -> None:
    """
    Handle changes in the dataset dropdown selection.

    Parameters
    ----------
    change : dict
        The change event dictionary.
    """
    with selection_output:
        clear_output(wait=True)
        file_upload.value.clear()
        file_upload._counter = 0
        status_label.value = ""
        submit_button.disabled = False
        if dataset_dropdown.value == "custom":
            display(file_upload)


dataset_dropdown.observe(on_dataset_change, names="value")

def on_submit_clicked(b: Button) -> None:
    """
    Handle the submit button click event for dataset selection and year validation.

    Parameters
    ----------
    b : Button
        The button widget that was clicked.
    """
    global start_time, end_time, mean_mb_ds, dataset_url
    with selection_output:
        clear_output(wait=False)
    submit_button.disabled = True
    status_label.value = "⏳ Processing, please wait..."
    # Define allowed year ranges
    dataset_ranges = {
        "GrIS": (1992, 2019),
        "AIS": (2002, 2025),
        "AIS and GrIS": (2002, 2019),
        "custom": (None, None),  # No restriction for custom
    }
    dataset_name = dataset_dropdown.value
    min_year, max_year = dataset_ranges.get(dataset_name, (None, None))
    start_year = start_year_widget.value
    end_year = end_year_widget.value

    validation_error = None
    # Validate years
    if min_year is not None and (start_year < min_year or end_year > max_year):
        validation_error = f"Error: For {dataset_name}, years must be between {min_year} and {max_year}."
    if start_year > end_year:
        validation_error = "Error: Start year must be less than or equal to end year."
    if not (isinstance(start_year, int) and isinstance(end_year, int)):
        validation_error = "Error: Years must be integers."
    if dataset_name=="custom" and not file_upload.value:
        validation_error = "Error: Please upload a file."
    if validation_error:
        status_label.value = validation_error
        submit_button.disabled = False
        return
    with selection_output:
        clear_output(wait=True)

        start_time = datetime(start_year, 1, 1)
        end_time = datetime(end_year, 12, 31)
        dataset_to_api_name = {
            "GrIS": "greenland",
            "AIS": "antarctic",
            "AIS and GrIS": "greenland_and_antarctic",
        }
        dataset_url = upload_mass_balance_csv(next(iter((file_upload.value.values())))) if dataset_name == "custom" \
            else get_precomputed_mass_balance_dataset_url(dataset_to_api_name[dataset_name])
        mass_balance_ds = xr.open_dataset(dataset_url, engine="zarr")
        mean_mb_ds = compute_mean_mass_balance_over_time_window(mass_balance_ds, start_time, end_time)
        status_label.value = ""
        submit_button.disabled = False

        print(f"Selected dataset: {dataset_dropdown.label if hasattr(dataset_dropdown, 'label') else dataset_name}")
        print(f"Start year: {start_time.year}")
        print(f"End year: {end_time.year}")
        print(f"Dataset URL: {dataset_url}")

submit_button.on_click(on_submit_clicked)

display(dataset_dropdown, start_year_widget, end_year_widget, submit_button, selection_output, status_label)

## True mass-balance trajectories and exploring "what-if" scenarios

Now that you've selected a mass input dataset, here is a visualization of the spatial pattern of mass change for your chosen region and time period.

- If you want to explore the **default scenario** (i.e., the observed or provided mass change), simply continue with the scaling factor set to **1**.
- If you are interested in exploring **"what-if" scenarios**, you can adjust the scaling factor below.  
  For example:  
  *What if we had double the amount of ice melt over the same period?* → set the scale factor to **2**.

This allows you to investigate a range of possible future or hypothetical mass input scenarios and see how they would affect sea level.

*Hint: changing the scale factor takes a few seconds to compute. Please be patient.*

In [ ]:
total_mb_gt = mean_mb_ds[MASS_BALANCE_COL_NAME].sum() / 1e12
total_mb_err_gt = mean_mb_ds[MASS_BALANCE_ERROR_COL_NAME].sum(skipna=True) / 1e12

plot_output = widgets.Output()
scale_input = widgets.FloatText(value=1.0, description="Scaler:", step=0.01)

def on_scale_change(change: dict) -> None:
    """
    Handle changes to the scaling factor input and update the plot accordingly.

    Parameters
    ----------
    change : dict
        The change event dictionary from the widget.
    """
    global mb_str
    with plot_output:
        clear_output(wait=True)
        mb_str = f"MB: {scale_input.value * total_mb_gt:.1f} Gt/yr ± {scale_input.value * total_mb_err_gt:.1f} Gt/yr " \
                 f"| Scaled: {scale_input.value:.2f}x."
        plot_scaled_mean_mass_balance(
            mean_mb_ds,
            dataset_value=dataset_dropdown.value,
            plot_description_str=\
                f"Mass input from {dataset_dropdown.value} dataset ({start_time.year}-{end_time.year})\n{mb_str}",
            scale=scale_input.value,
        )

scale_input.observe(on_scale_change, names="value")
display(scale_input, plot_output)

# Trigger the plot once at startup, with scale factor = 1.0 as default
with plot_output:
    mb_str = f"MB: {scale_input.value * total_mb_gt:.1f} ± {scale_input.value * total_mb_err_gt:.1f} Gt/yr " \
             f"| Scaled: {scale_input.value:.2f}x"
    plot_scaled_mean_mass_balance(
        mean_mb_ds,
        dataset_value=dataset_dropdown.value,
        plot_description_str=\
            f"Mass input from {dataset_dropdown.value} dataset ({start_time.year}-{end_time.year})\n{mb_str}",
        scale=scale_input.value)

## How does the mass input translate to sea-level change globally?

After importing, and if optional **scaling**, the **terrestrial mass contribution**  
(e.g., from ice sheets, glaciers, and terrestrial water storage) to the oceans,  
we can now examine the **instantaneous Equivalent Sea Level (ESL)**.

- The **instantaneous global equivalent sea level response** is calculated based on your selected scaling factor and the input data (i.e., the cumulated mass balance for the chosen period and region).
- The **relative sea-level change** at each location is computed as the difference between the absolute sea-level change and the local vertical deformation (e.g., due to glacial isostatic adjustment).
- The calculation also accounts for **rotational feedback**, which describes how the redistribution of water mass is affected by Earth"s rotation and, consequently, the spatial pattern of sea-level change.
- The **uncertainty** shown here reflects the combined uncertainty from the absolute sea-level change and the vertical deformation components, but does **not** include the uncertainty from the original mass balance input data.

---

In [ ]:
status_label = widgets.Label(value="")
display(status_label)

status_label.value = "⏳ Running SELREM module (this may take a few minutes)..."
global_slr_ds = run_selrem_module(dataset_url, scale_input.value, start_time.year, end_time.year, "global")
suptitle = \
    f"Global sea-level response\nMass input from {dataset_dropdown.value} ({start_time.year}-{end_time.year})\n{mb_str}"
status_label.value = "⏳ Generating plot..."
plot_global_slr_three_panels(
    global_slr_ds,
    mean_mb_ds,
    plot_description_str=suptitle
)
status_label.value = ""

## Explore the sea-level response at a chosen location

In this section, you can investigate how sea level has changed over time at any location of your choice.

- **Specify coordinates:** Enter the latitude and longitude for the location you are interested in.
- **Adjust scenarios:** You can again modify the scaling factor to explore different "what-if" scenarios for mass input.
- **Visualize results:** The resulting plot will show the time series of sea-level change at your chosen location based on the selected dataset and scenario.

This allows you to examine the local impact of terrestrial mass changes on the local sea level, and to compare how different scenarios might affect specific regions around the globe.

In [ ]:
lat_input = widgets.FloatText(value=55.7, description="Latitude:", step=0.1)
lon_input = widgets.FloatText(value=12.6, description="Longitude:", step=0.1)
scale_loc_input = widgets.FloatText(value=1.0, description="Scaler:", step=0.01)
plot_button = widgets.Button(description="Plot SLR at location", button_style="success")
note_label = widgets.Label(value="Note: When changing the scaling factor, " \
                           "plotting may take a few minutes to recalculate.")
status_label = widgets.Label(value="⏳ Running SELREM module (this may take a few minutes)...")
slr_plot_output = widgets.Output()

def on_plot_button_clicked(b: Button) -> None:
    """
    Handle the plot button click event to display sea-level response at the selected location.

    Parameters
    ----------
    b : Button
        The button widget that was clicked.
    """
    lat0 = lat_input.value
    lon0 = lon_input.value
    with slr_plot_output:
        clear_output(wait=True)
        status_label.value = "Generating plot..."
        plot_button.disabled = True
        plot_slr_location_four_panels(annual_slr_ds, lat0=lat0, lon0=lon0)
        status_label.value = ""
        plot_button.disabled = False

plot_button.on_click(on_plot_button_clicked)

def on_scale_loc_change(change: dict) -> None:
    """
    Handle changes to the scaling factor input for the location-specific SLR plot.

    Parameters
    ----------
    change : dict
        The change event dictionary from the widget.
    """
    global annual_slr_ds
    status_label.value = "⏳ Running SELREM module (this may take a few minutes)..."
    plot_button.disabled = True
    annual_slr_ds = run_selrem_module(dataset_url, scale_input.value, start_time.year, end_time.year, "annual")
    status_label.value = ""
    plot_button.disabled = False


scale_loc_input.observe(on_scale_loc_change, names="value")
plot_button.disabled = True
display(note_label, lat_input, lon_input, scale_loc_input, plot_button, status_label, slr_plot_output)
annual_slr_ds = run_selrem_module(dataset_url, scale_input.value, start_time.year, end_time.year, "annual")
status_label.value = ""
plot_button.disabled = False